# fMRI Analysis 2018
For single scans (have to be in .BRIK format already). Needs AFNI installed.

**Usage**: Adjust the settings below.<br> 
Run this script ("*Run*" button above, or CTRL+ENTER to run cell-by-cell). <br>
Popup will open --> select the scan. <br>
Results are stored in ./analyze subfolder

## 1. Important settings

In [1]:
TR = 1 # in seconds

initial_cutoff = 10 # number of volumes to remove (in absence of dummy scans).

blurFWHM = 0.3 # In [mm]. Rule-of-thumb: 1.5x the voxel resolution.

highpass = 200 # period [s]. Any signal slower than this will be removed. Should be at least 2x inter-stim. interval


# --- For averaging of blocks:

averaging_baseline = 20 # baseline to consider before each block

_time = 60 # how many seconds to display after stim block ends


# --- Location of an EPI template for coregistration (should be a representative animal with same geometry settings)

template_path = '/home/aic/BigHDD/MR_DATA/CSI2018/template/TEMPLATE+orig'


# --- Define the stimulation paradigm:

stim_duration = 40  # everything in seconds
baseline = 12*60
Nblocks = 10
ISI = 60  # Inter-stimulus interval (on+off period)


#----
stimList=list(range(baseline,baseline*Nblocks,ISI))
stimString=' '.join(map(str,stimList))
stimulus_times = "'1D: "+stimString+"'"
print('stimulus times:',stimList)

stimulus times: [720, 780, 840, 900, 960, 1020, 1080, 1140, 1200, 1260, 1320, 1380, 1440, 1500, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2040, 2100, 2160, 2220, 2280, 2340, 2400, 2460, 2520, 2580, 2640, 2700, 2760, 2820, 2880, 2940, 3000, 3060, 3120, 3180, 3240, 3300, 3360, 3420, 3480, 3540, 3600, 3660, 3720, 3780, 3840, 3900, 3960, 4020, 4080, 4140, 4200, 4260, 4320, 4380, 4440, 4500, 4560, 4620, 4680, 4740, 4800, 4860, 4920, 4980, 5040, 5100, 5160, 5220, 5280, 5340, 5400, 5460, 5520, 5580, 5640, 5700, 5760, 5820, 5880, 5940, 6000, 6060, 6120, 6180, 6240, 6300, 6360, 6420, 6480, 6540, 6600, 6660, 6720, 6780, 6840, 6900, 6960, 7020, 7080, 7140]


In [2]:
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import subprocess
import shutil
import sys
import numpy as np
import glob

## 2. Selecting scan

In [3]:
root = Tk()
ftypes = [('EPI Scan',"*.BRIK*"),("All Files","*.*")]
ttl = "Select file"
dir1 = '/home/aic/BigHDD'
    
root.withdraw()
file_path = askopenfilename(filetypes = ftypes, initialdir = dir1, title = ttl)
root.update()
root.destroy()

data_path = os.path.dirname(file_path)
scanname=os.path.splitext(os.path.basename(file_path))[0][:-5]

new_subfolder = 'analyzed'
newfolder = os.path.join(os.path.expanduser(data_path),new_subfolder)

print('loaded '+file_path)
os.chdir(data_path)

loaded /BigHDD/MR_DATA/CSI2018/20180620_112234_CSI_BOLD_Elec_Stim_1mA_1_3/6/pdata/1/E6+orig.BRIK


In [4]:
def runAFNI(inputstring):
    print(inputstring)
    subprocess.run(inputstring,shell=True, check=True)

## 3. Coregistration & Motion correction

In [5]:
runAFNI('3dTcat -prefix '+ newfolder + '/' + scanname + ' ' + scanname + "+orig'["+ str(initial_cutoff) + "..$]'")
runAFNI('3dTcat -prefix '+ newfolder + '/single_timepoint' + ' ' + scanname + "+orig'[0]'")

os.chdir(newfolder)

runAFNI("python2.7 /home/aic/abin/align_epi_anat.py -dset2to1 -dset1 " + template_path + " -dset2 single_timepoint+orig" \
                                " -child_dset2 " + scanname + '+orig ' \
                                " -dset1_strip None -dset2_strip None"  \
                                " -overwrite" \
                                " -big_move" \
                                " -volreg on -volreg_method 3dvolreg" \
                                " -volreg_opts '-Fourier -zpad 1'" \
                                " -Allineate_opts '-maxrot 10 -maxshf 3 -conv 0.005 -twofirst -twoblur 0.8 -source_automask+2 -final wsinc5'" \
                                " -tshift on -tshift_opts '-tzero 0 -quintic'" \
                                " -dset2_base 0" \
                                " -volreg_base 0" \
                                " -suffix .volreg" \
                                " -cost ls")
    
runAFNI('3dTstat -prefix rm.mean' + ' ' + scanname + '.volreg+orig')
runAFNI('3dcalc -a ' + scanname + '.volreg+orig -b rm.mean+orig'\
                    " -expr 'min(200, a/b*100)*step(a)*step(b)' -prefix r" + scanname + '.scale')

3dTcat -prefix /BigHDD/MR_DATA/CSI2018/20180620_112234_CSI_BOLD_Elec_Stim_1mA_1_3/6/pdata/1/analyzed/E6 E6+orig'[10..$]'
3dTcat -prefix /BigHDD/MR_DATA/CSI2018/20180620_112234_CSI_BOLD_Elec_Stim_1mA_1_3/6/pdata/1/analyzed/single_timepoint E6+orig'[0]'
python2.7 /home/aic/abin/align_epi_anat.py -dset2to1 -dset1 /home/aic/BigHDD/MR_DATA/CSI2018/template/TEMPLATE+orig -dset2 single_timepoint+orig -child_dset2 E6+orig  -dset1_strip None -dset2_strip None -overwrite -big_move -volreg on -volreg_method 3dvolreg -volreg_opts '-Fourier -zpad 1' -Allineate_opts '-maxrot 10 -maxshf 3 -conv 0.005 -twofirst -twoblur 0.8 -source_automask+2 -final wsinc5' -tshift on -tshift_opts '-tzero 0 -quintic' -dset2_base 0 -volreg_base 0 -suffix .volreg -cost ls
3dTstat -prefix rm.mean E6.volreg+orig
3dcalc -a E6.volreg+orig -b rm.mean+orig -expr 'min(200, a/b*100)*step(a)*step(b)' -prefix rE6.scale


## 4. Preprocessing
Detrending/Filtering/Blurring

In [7]:
runAFNI('3dTproject -input r*.scale+orig.BRIK -passband ' + str(1/highpass) + ' 99999 -blur ' + str(blurFWHM) + ' -prefix CleanedData')

3dTproject -input r*.scale+orig.BRIK -passband 0.005 99999 -blur 0.3 -prefix CleanedData


## 5. GLM analysis

In [8]:
runAFNI('3dDeconvolve -input CleanedData+orig' \
                    ' -polort 0' \
                    ' -nodmbase' \
                    ' -overwrite' \
                    ' -num_stimts 1' \
                    ' -stim_times 1 ' +  stimulus_times + \
                    " 'SPMG3(" + str(stim_duration) + ")'" \
                    ' -stim_times_subtract ' + str(initial_cutoff*TR) + \
                    ' -stim_label 1 electro' \
                    ' -iresp 1 HRF_SPM_el' \
                    ' -fout -tout -x1D XSPM.xmat.1D -xjpeg XSPM.jpg' \
                    ' -fitts fittsSPM_el' \
                    ' -errts errtsSPM_el' \
                    ' -bucket statsSPM_el' \
                    ' -cbucket regcoeffsSPM_el')

3dDeconvolve -input CleanedData+orig -polort 0 -nodmbase -overwrite -num_stimts 1 -stim_times 1 '1D: 720 780 840 900 960 1020 1080 1140 1200 1260 1320 1380 1440 1500 1560 1620 1680 1740 1800 1860 1920 1980 2040 2100 2160 2220 2280 2340 2400 2460 2520 2580 2640 2700 2760 2820 2880 2940 3000 3060 3120 3180 3240 3300 3360 3420 3480 3540 3600 3660 3720 3780 3840 3900 3960 4020 4080 4140 4200 4260 4320 4380 4440 4500 4560 4620 4680 4740 4800 4860 4920 4980 5040 5100 5160 5220 5280 5340 5400 5460 5520 5580 5640 5700 5760 5820 5880 5940 6000 6060 6120 6180 6240 6300 6360 6420 6480 6540 6600 6660 6720 6780 6840 6900 6960 7020 7080 7140' 'SPMG3(40)' -stim_times_subtract 10 -stim_label 1 electro -iresp 1 HRF_SPM_el -fout -tout -x1D XSPM.xmat.1D -xjpeg XSPM.jpg -fitts fittsSPM_el -errts errtsSPM_el -bucket statsSPM_el -cbucket regcoeffsSPM_el


## 6. Averaging stimulation blocks
For plotting 

In [9]:
k=1
for i in stimList-initial_cutoff:
    runAFNI('3dTcat -prefix temp_average_raw' + str(k) + ' CleanedData+orig[' + str(int(i) - averaging_baseline ) + '..' + str(int(i) + stim_duration + _time) +']')
    k += 1

    
runAFNI('3dTcat -prefix average_spm' + ' fittsSPM_el+orig[' + str(stimList[0] - averaging_baseline ) + '..' + str(stimList[0] + stim_duration + _time) +']')
runAFNI('3dMean -prefix average_raw temp_average_raw*')

runAFNI("bash -c 'rm -f temp_average* &>/dev/null'")


TypeError: unsupported operand type(s) for -: 'list' and 'int'